## Generating Synthetic Data 

**Objective:**
Generate synthetic data for various tables related to KPIs for Pro Juventute.

### **Tables and Constraints**

**Circles Table:**

* Contains unique circle names derived from sample data.
* Timestamps for creation and last update.

**Users Table:**

* Generate data for 10 users.
* Users can belong to any circle.
* Each user has a unique username and email.
* Users have roles, either 'Admin' or 'User'.
* Timestamps for account creation and last update.

**KPIs Table:**

* Generate data for KPIs related to Pro Juventute's activities.
* KPIs can occasionally be shared across circles.
* Each KPI has a unique name, description, periodicity (Daily, Weekly, etc.), and unit of measurement.
* KPIs that represent percentages have a maximum value of 100.
* KPIs that do not represent percentages do not have a predefined maximum value (e.g., funds raised, sessions, etc.).
* Timestamps for KPI creation and last update.

**KPI Values Table:**

* Generate values for each KPI for 3 years.
* The values lie between the minimum and maximum specified for each KPI.
* Timestamps for each value's creation and last update.

**Audit Logs Table:**

* Generate log entries for each KPI value.
* Each log entry has an action type (Create, Update, Delete), table name, record name, user ID, and timestamp.

**KPI Targets Table:**

* Generate target values for each KPI.
* Targets have a timeframe (Daily, Weekly, etc.) and are associated with specific KPIs.
* Timestamps for target creation and last update.

**Additional Information:**

The generated data will be exported to .csv files for each table. The KPIs and their descriptions related to Pro Juventute's activities are provided in a list of dictionaries format. The sample data provided contains some KPI names and their acceptable ranges or descriptions.

Generation with assistance from ChatGPT.

In [40]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os

random.seed(1)

#sample_size = 

sample_data_path = '../sample-data/pj_sample_value.csv'

synthetic_data_dir = '../synthetic-data/'

# Load the sample data
sample_data_df = pd.read_csv(sample_data_path)

# Generate Circles Table data
circles_df = pd.DataFrame({
    'circle_id': range(1, sample_data_df['circle'].nunique() + 1),
    'circle_name': sample_data_df['circle'].unique(),
    'created_at': [datetime.now() - timedelta(days=random.randint(0, 365*3)) for _ in range(sample_data_df['circle'].nunique())],
    'updated_at': [datetime.now() - timedelta(days=random.randint(0, 365*2)) for _ in range(sample_data_df['circle'].nunique())]
})
circles_df.to_csv(os.path.join(synthetic_data_dir, 'circles.csv'), index=False)

# Generate Users Table data (10 users as an example)
num_users = 10
users_df = pd.DataFrame({
    'user_id': range(1, num_users+1),
    'circle_id': [random.choice(circles_df['circle_id'].tolist()) for _ in range(num_users)],
    'username': [f"User_{i}" for i in range(1, num_users+1)],
    'email': [f"user{i}@example.com" for i in range(1, num_users+1)],
    'password': ['password'] * num_users,
    'role': ['Admin' if i == 0 else 'User' for i in range(num_users)],
    'created_at': [datetime.now() - timedelta(days=random.randint(0, 365*3)) for _ in range(num_users)],
    'updated_at': [datetime.now() - timedelta(days=random.randint(0, 365*2)) for _ in range(num_users)]
})
users_df.to_csv(os.path.join(synthetic_data_dir, 'users.csv'), index=False)

# Define KPIs related to Pro Juventute's activities

# mapping the enum as an integer 
unit_mapping = {
    'numeric': 2,
    '%': 0,
    'boolean': 1
}

pro_juventute_kpis = [
    {"kpi_name": "Number of counseling sessions", "description": "Total counseling sessions held in a period.", "unit": None},
    {"kpi_name": "Active volunteers", "description": "Number of active volunteers for Pro Juventute.", "unit": None},
    {"kpi_name": "Workshops conducted", "description": "Total number of workshops conducted.", "unit": None},
    {"kpi_name": "Funds raised", "description": "Total funds raised in a period.", "unit": "numeric"},
    {"kpi_name": "Children reached", "description": "Number of children reached through programs.", "unit": None},
    {"kpi_name": "Digital literacy programs", "description": "Number of digital literacy programs conducted.", "unit": None},
    {"kpi_name": "Community events", "description": "Number of community events organized.", "unit": None},
    {"kpi_name": "Feedback received", "description": "Number of feedback entries received from beneficiaries.", "unit": None},
    {"kpi_name": "Outreach programs", "description": "Total outreach programs conducted.", "unit": None},
    {"kpi_name": "Online safety workshops", "description": "Number of online safety workshops held.", "unit": None},
    {"kpi_name": "Partnerships formed", "description": "Number of new partnerships or collaborations.", "unit": None},
    {"kpi_name": "Awareness campaigns", "description": "Total awareness campaigns conducted.", "unit": None}
]

# Generate KPIs Table data
kpis_data = []
for idx, circle in circles_df.iterrows():
    for kpi in pro_juventute_kpis:
        kpis_data.append({
            'kpi_id': len(kpis_data) + 1,
            'kpi_name': kpi['kpi_name'],
            'description': kpi['description'],
            'periodicity': random.choice(['daily', 'weekly', 'monthly', 'quarterly', 'yearly']),
            'value_min': 0,
            'value_max': 100 if kpi['unit'] == '%' else None,
            'unit': kpi['unit'],
            'created_at': datetime.now() - timedelta(days=random.randint(0, 365*3)),
            'updated_at': datetime.now() - timedelta(days=random.randint(0, 365*2))
        })

kpis_df = pd.DataFrame(kpis_data)

# note to remove once the 'unit' datatype issue is resovled 

kpis_df['unit'] = np.nan

kpis_df.to_csv(os.path.join(synthetic_data_dir, 'kpi_definition.csv'), index=False)

# Generate KPI_Values Table data without anomalies
num_kpi_values = len(sample_data_df) * 3
kpi_values_df = pd.DataFrame({
    'kpi_value_id': range(1, num_kpi_values + 1),
    'kpi_id': [random.choice(kpis_df['kpi_id'].tolist()) for _ in range(num_kpi_values)],
    'cirlce_id': [random.choice(circles_df['circle_id'].tolist()) for _ in range(num_kpi_values)],
    'user_id': [random.choice(users_df['user_id'].tolist()) for _ in range(num_kpi_values)],
    'value': [random.uniform(0, 100) for _ in range(num_kpi_values)],
    'period_year': [(datetime.now() - timedelta(days=random.randint(0, 365*3))).year for _ in range(num_kpi_values)],
    'period_month': [random.randint(1, 12) for _ in range(num_kpi_values)],
    'created_at': [datetime.now() - timedelta(days=random.randint(0, 365*3)) for _ in range(num_kpi_values)],
    'updated_at': [datetime.now() - timedelta(days=random.randint(0, 365*2)) for _ in range(num_kpi_values)]
})

# Introduce anomalies to the KPI Values table
for _ in range(2):
    idx = random.choice(kpi_values_df.index)
    kpi_values_df.at[idx, 'value'] *= random.choice([10, 0.1])

for _ in range(3):
    idx = random.choice(kpi_values_df.index)
    kpi_values_df.at[idx, 'value'] = random.choice([-500, 5000])

kpi_values_df.to_csv(os.path.join(synthetic_data_dir, 'kpi_values.csv'), index=False)

# Generate Audit_Logs Table data
num_audit_logs = len(kpi_values_df)
audit_logs_df = pd.DataFrame({
    'session_id': range(1, num_audit_logs + 1),
    'action': ['Create'] * num_audit_logs,  # Assuming all are create actions for simplicity
    'table_name': ['KPI_Values'] * num_audit_logs,
    'record_name': kpi_values_df['kpi_value_id'].tolist(),
    'user_id': kpi_values_df['user_id'].tolist(),
    'timestamp': kpi_values_df['created_at'].tolist()
})

audit_logs_df.to_csv(os.path.join(synthetic_data_dir, 'audit_logs.csv'), index=False)

# Generate KPI_Targets Table data

# Generate random future dates for each target
future_dates = [datetime.now() + timedelta(days=random.randint(1, 365)) for _ in range(num_kpi_targets)]

# number of kpis
num_kpi_targets = len(kpis_df)

# Predefined list of nice, round target values
nice_round_numbers = [100, 200, 400, 800, 1000, 2000, 4000, 8000, 10000, 20000, 40000, 80000, 100000]

# Randomly select target values from the list of nice, round numbers
target_values = [random.choice(nice_round_numbers) for _ in range(num_kpi_targets)]


kpi_targets_df = pd.DataFrame({
    'target_id': range(1, num_kpi_targets + 1),
    'kpi_id': kpis_df['kpi_id'].tolist(),
    'target_value': target_values,
    # 'unit': kpis_df['unit'].tolist(),
    'timeframe': future_dates,
    'created_at': [datetime.now() - timedelta(days=random.randint(0, 365*3)) for _ in range(num_kpi_targets)],
    'updated_at': [datetime.now() - timedelta(days=random.randint(0, 365*2)) for _ in range(num_kpi_targets)]
})



kpi_targets_df.to_csv(os.path.join(synthetic_data_dir, 'target.csv'), index=False)


print("Data generation complete")


Data generation complete


In [12]:
audit_logs_df.head()


,session_id,action,table_name,record_name,user_id,timestamp
0,1,Create,KPI_Values,1,3,2022-02-27 15:19:50.039051
1,2,Create,KPI_Values,2,10,2022-04-08 15:19:50.039052
2,3,Create,KPI_Values,3,5,2022-06-05 15:19:50.039053
3,4,Create,KPI_Values,4,8,2022-07-24 15:19:50.039054
4,5,Create,KPI_Values,5,9,2023-03-10 15:19:50.039054


In [18]:
circles_df.head()

,circle_id,circle_name,created_at,updated_at
0,1,HR,2022-12-27 15:42:24.700705,2022-06-02 15:42:24.700753
1,2,Programs - Children - Counceling,2023-05-22 15:42:24.700726,2021-11-30 15:42:24.700755
2,3,Programs - Parents -Online,2022-04-24 15:42:24.700728,2022-09-05 15:42:24.700756
3,4,Fundraising,2023-01-30 15:42:24.700729,2023-02-26 15:42:24.700757
4,5,Digital,2020-12-18 15:42:24.700730,2023-06-24 15:42:24.700758


In [31]:
kpis_df['unit'].value_counts()

kpis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   kpi_id       72 non-null     int64         
 1   kpi_name     72 non-null     object        
 2   description  72 non-null     object        
 3   periodicity  72 non-null     object        
 4   value_min    72 non-null     int64         
 5   value_max    0 non-null      object        
 6   unit         72 non-null     object        
 7   created_at   72 non-null     datetime64[ns]
 8   updated_at   72 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 5.2+ KB


In [13]:
kpi_targets_df.head()

,target_id,kpi_id,target_value,timeframe,created_at,updated_at
0,1,1,1979.180723,Quarterly,2022-09-27 15:19:50.044221,2023-09-14 15:19:50.044282
1,2,2,2246.238409,Quarterly,2021-04-11 15:19:50.044224,2022-08-28 15:19:50.044283
2,3,3,4163.272212,Monthly,2021-03-22 15:19:50.044225,2022-05-18 15:19:50.044284
3,4,4,4855.374683,Yearly,2020-10-01 15:19:50.044226,2022-06-29 15:19:50.044284
4,5,5,1214.477831,Weekly,2021-04-06 15:19:50.044227,2022-11-24 15:19:50.044285


In [36]:
kpi_values_df.head()

,kpi_value_id,kpi_id,cirlce_id,user_id,value,period_year,period_month,created_at,updated_at
0,1,6,4,3,53.212840,2023,11,2022-02-27 16:30:00.505006,2023-02-07 16:30:00.505223
1,2,11,4,10,16.779754,2023,2,2022-04-08 16:30:00.505007,2021-11-17 16:30:00.505224
2,3,18,4,5,14.835499,2022,3,2022-06-05 16:30:00.505008,2023-03-23 16:30:00.505225
3,4,22,2,8,68.724220,2021,11,2022-07-24 16:30:00.505009,2023-05-29 16:30:00.505226
4,5,22,3,9,56.277553,2021,1,2023-03-10 16:30:00.505010,2023-08-01 16:30:00.505227
